<a href="https://colab.research.google.com/github/apetros/pyeplan/blob/master/examples/1_bus_Planning_Example_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Illustrative example for optimizing investment and operation of a microgrid considering all generation and load connected to single bus

This simple example neglects the network model and considers a single node with total load. One representative day with is used to describe the potential operational scenarios. Three types of investment candidate generators i.e, wind units, solar units and diesel generation units. It showcases how the Investment and Operation Planning Modules of PyEPLAN can be used to derive optimal units to be installed in the microgrid network.

This example is designed to be run on Google Colab.

In [24]:
!rm -r sample_data
!rm -r 1_bus

rm: cannot remove 'sample_data': No such file or directory


In [25]:
!apt-get install -qq subversion
!svn export https://github.com/apetros/pyeplan/trunk/examples/1_bus

A    1_bus
A    1_bus/cbat_dist.csv
A    1_bus/cblt_dist.csv
A    1_bus/cgen_dist.csv
A    1_bus/clin_dist.csv
A    1_bus/csol_dist.csv
A    1_bus/cwin_dist.csv
A    1_bus/dtim_dist.csv
A    1_bus/egen_dist.csv
A    1_bus/elin_dist.csv
A    1_bus/esol_dist.csv
A    1_bus/ewin_dist.csv
A    1_bus/geol_dist.csv
A    1_bus/mgpc_dist.xlsx
A    1_bus/pdem_dist.csv
A    1_bus/prep_dist.csv
A    1_bus/psol_dist.csv
A    1_bus/pwin_dist.csv
A    1_bus/qdem_dist.csv
A    1_bus/qrep_dist.csv
A    1_bus/qsol_dist.csv
A    1_bus/qwin_dist.csv
A    1_bus/results
A    1_bus/results/cu.csv
A    1_bus/results/eu.csv
A    1_bus/results/obj.csv
A    1_bus/results/pbc.csv
A    1_bus/results/pbd.csv
A    1_bus/results/pcg.csv
A    1_bus/results/pcs.csv
A    1_bus/results/pcw.csv
A    1_bus/results/pds.csv
A    1_bus/results/peg.csv
A    1_bus/results/pel.csv
A    1_bus/results/pes.csv
A    1_bus/results/pew.csv
A    1_bus/results/pss.csv
A    1_bus/results/pws.csv
A    1_bus/results/qcd.csv
A    1_bus/res

In [26]:
!apt-get install -y -qq glpk-utils

In [27]:
!apt-get install -y -qq coinor-cbc

In [28]:
!pip install -q pyeplan

In [29]:
!pip install git+git://github.com/apetros/pyeplan@master

  Cloning git://github.com/apetros/pyeplan (to revision master) to /tmp/pip-req-build-h313str3
  Running command git clone -q git://github.com/apetros/pyeplan /tmp/pip-req-build-h313str3
  Created wheel for pyeplan: filename=pyeplan-0.4.3-py3-none-any.whl size=15846 sha256=39f0273a1f4884d57656cc59b25c2974838af57f9a62c9ee702e01d19bfee091
  Stored in directory: /tmp/pip-ephem-wheel-cache-waag1_m_/wheels/9e/05/40/2374125e8f55142a752f54005fbe165970a6dda55bb008fbf0
Successfully built pyeplan
  Attempting uninstall: pyeplan
    Found existing installation: pyeplan 0.4.2
    Uninstalling pyeplan-0.4.2:
      Successfully uninstalled pyeplan-0.4.2


## Defining the input data

The next step is to define the directory to the folder where the input data is located and define the reference node. The input data folder should consists of 'csv' files that contain data description of the load, newtork paramters and generation units as defined [here](https://pyeplan.sps-lab.org/user_guide/input.html#).

Some of the data inputs include:

### Defining the total load demand at each hour

The total active power consumption at each of the three (3) hours for one (1) representative day is:

In [30]:
import pandas as pd
pd.read_csv("1_bus/prep_dist.csv")

,0
0,90


### Defining the input investment candidates

#### Wind Units

In [31]:
pd.read_csv("1_bus/cwin_dist.csv")

,bus,icost,ocost,scost,pmin,pmax,qmin,qmax
0,0,186,0,0,0,50,0,0
1,0,186,0,0,0,50,0,0


#### Solar PV Units

In [32]:
pd.read_csv("1_bus/csol_dist.csv")

,bus,icost,ocost,scost,pmin,pmax,qmin,qmax
0,0,109,0,0,0,50,0,0
1,0,109,0,0,0,50,0,0


#### Diesel/Fossil Units

In [33]:
pd.read_csv("1_bus/cgen_dist.csv")

,bus,icost,ocost,scost,pmin,pmax,qmin,qmax,hr
0,0,12,0.4,0,0,0,0,0,NaN
1,0,12,0.4,0,0,0,0,0,NaN


## Case studies
Case 1: No diesel candidates

Case 2: Solar + Wind + diesel candidated

Case 3: Total load reduced from 90 kW to 70 kW

## Using the invesment and operation planning module from PyEPLAN 

In [34]:
import pyeplan as pye

The module is initialised with inputs including but not limited the following :
* The input directory for the data. The input data folder should consists of 'csv' files that contain data description of the load, newtork paramters and generation units as defined [here](https://pyeplan.sps-lab.org/user_guide/input.html#).
* ref_bus: Reference node
* dshed_cost: Demand Shedding Price
* rshed_cost: Renewable Shedding Price
* phase: Number of Phases (Default 3)
* sbase: Base Apparent Power (Default 1 kW)

In [35]:
inosys = pye.inosys('1_bus', ref_bus = 0, dshed_cost = 100, rshed_cost = 0, phase = 3, )

## Solving the optimization problem

PyEPLAN can be used to solve the problem investment and operation planning problems simultaneously. In case no investment candidates are availble, a sole operation planning problem can be run by setting input 'onlyopr = True'. Available solvers inclde both open source solvers include glpk, cbc, and commercial solvers ipopt, gurobi given one has the required licences. If discrete capacities of investment units are available the input 'invest = True' this sets the investement-related decision variables to a binary nature where the unit capacity = nominal. Otherwise,  'invest = False' sets the investement-related decision variables to a continuous nature where the unit capacity <= nominal.

In [36]:
inosys.solve(solver = 'glpk', onlyopr = False, invest = True, )

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmptwqm2baa.glpk.raw --wglp /tmp/tmpkas_8eer.glpk.glp --cpxlp
 /tmp/tmprggumewq.pyomo.lp
Reading problem data from '/tmp/tmprggumewq.pyomo.lp'...
/tmp/tmprggumewq.pyomo.lp:248: warning: lower bound of variable 'xg(0)' redefined
/tmp/tmprggumewq.pyomo.lp:248: warning: upper bound of variable 'xg(0)' redefined
42 rows, 31 columns, 81 non-zeros
6 integer variables, all of which are binary
254 lines were read
Writing problem data to '/tmp/tmpkas_8eer.glpk.glp'...
217 lines were written
GLPK Integer Optimizer, v4.65
42 rows, 31 columns, 81 non-zeros
6 integer variables, all of which are binary
Preprocessing...
9 rows, 14 columns, 28 non-zeros
6 integer variables, all of which are binary
Scaling...
 A: min|aij| =  3.333e-01  max|aij| =  4.000e+01  ratio =  1.200e+02
GM: min|aij| =  7.598e-01  max|aij| =  1.316e+00  ratio =  1.732e+00
EQ: min|aij| =  5.774e-01  max|aij| =  1.000e+00  ratio =  1.732e+00

## Results

A folder named 'results' will be created with the output of the optimal solution to the planning problem. The different result files are defined [here](https://pyeplan.sps-lab.org/user_guide/output.html). Below we show the capital costs and operational costs obtained to satify the load in the 5-bus network.

### Total Investment and Operational Costs

In [37]:
pd.read_csv("1_bus/results/obj.csv")

,total costs,20200.0
0,total investment costs,20200.0
1,total operation costs,0.0


### Number and capacity of wind units installed

The capacity and location of wind units installed is:

In [39]:
import numpy as np
cwin = pd.read_csv("1_bus/cwin_dist.csv")
iwin = pd.read_csv("1_bus/results/xw.csv")
cwin['Unit'] = (np.arange(1,len(iwin.columns)+1))
unit = cwin.loc[:,'Unit']
bus = np.array(cwin.loc[:,'bus'])
out_win =(((cwin.loc[:,'pmax']*round(iwin.loc[0:,].T,2))[0]).to_frame().set_index(unit)).rename(columns={0: 'Installed Capacity (kW)'})
out_win['Bus'] = bus
out_win.style

,Installed Capacity (kW),Bus
Unit,,
1,50.000000,0
2,0.000000,0


### Number and capacity of solar units installed

The capacity and location of solar units installed is:

In [41]:
csol = pd.read_csv("1_bus/csol_dist.csv")
isol = pd.read_csv("1_bus/results/xs.csv")
csol['Unit'] = (np.arange(1,len(isol.columns)+1))
unit = csol.loc[:,'Unit']
bus = np.array(csol.loc[:,'bus'])
out_sol =(((csol.loc[:,'pmax']*round(isol.loc[0:,].T,2))[0]).to_frame().set_index(unit)).rename(columns={0: 'Installed Capacity (kW)'})
out_sol['Bus'] = bus
out_sol.style

,Installed Capacity (kW),Bus
Unit,,
1,50.000000,0
2,50.000000,0


### Number and capacity of diesel units installed

The capacity and location of diesel units installed is:

In [43]:
cgen = pd.read_csv("1_bus/cgen_dist.csv")
igen = pd.read_csv("1_bus/results/xg.csv")
cgen['Unit'] = (np.arange(1,len(igen.columns)+1))
unit = cgen.loc[:,'Unit']
bus = np.array(cgen.loc[:,'bus'])
out_gen =(((cgen.loc[:,'pmax']*round(igen.loc[0:,].T,2))[0]).to_frame().set_index(unit)).rename(columns={0: 'Installed Capacity (kW)'})
out_gen['Bus'] = bus
out_gen.style

,Installed Capacity (kW),Bus
Unit,,
1,0.000000,0
2,0.000000,0


### Amount of load curtailed

The level of load demand that has been curtailed at each node is:

In [44]:
pds =pd.read_csv("1_bus/results/pds.csv")
pds.index.name ='Hour'
pds.style

,0
Hour,
0,0.000000
